In [1]:
# !wget https://huggingface.co/datasets/mesolitica/translated-CodeUltraFeedback/resolve/main/translated-CodeUltraFeedback.jsonl

In [2]:
import json
import operator

In [3]:
results = []
with open('translated-CodeUltraFeedback.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if l['instruction_ms'] is None:
            continue
        
        try:
            responses = {}
            for r in l['responses']:
                if r['response_ms'] is not None:
                    responses[r['model']] = r['response_ms']
            
            annotations = {}
            for r in l['annotations']:
                if r['model'] in responses:
                    annotations[r['model']] = int(r['rating'])
        
                
            min_key = min(annotations.items(), key=operator.itemgetter(1))[0]
            max_key = max(annotations.items(), key=operator.itemgetter(1))[0]
            if annotations[min_key] == annotations[max_key]:
                continue
                
        except:
            continue
            
        if '```' not in responses[max_key]:
            continue
        
        results.append({
            'prompt': l['instruction_ms'],
            'chosen': responses[max_key],
            'rejected': responses[min_key]
        })
        
len(results)

7173

In [4]:
from datasets import Dataset

dataset = Dataset.from_list(results)

In [5]:
dataset.push_to_hub('mesolitica/DPO-translated-CodeUltraFeedback')

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Updating downloaded metadata with the new split.
